### Введение

Случайный лес — это модель классификации, объединяющая некоторое количество решающих деревьев в одну композицию, за счет чего улучшается их качество работы и обобщающая способность. Деревья строятся независимо друг от друга. Чтобы они отличались друг от друга, обучение проводится не на всей обучающей выборке, а на ее случайном подмножестве. Также, для дальнейшего уменьшения схожести деревьев, оптимальный признак для разбиения выбирается не из всех возможных признаков, а лишь из их случайного подмножества. Прогнозы, выданные деревьями, объединяются в один ответ путем усреднения.

Особенность случайного леса заключается в том, что он не переобучается по мере увеличения количества деревьев в композиции. Это достигается за счет того, что деревья не зависят друг от друга, и поэтому добавление нового дерева в композицию не усложняет модель, а лишь понижает уровень шума в прогнозах.

### Реализация в Scikit-Learn

В библиотеке `scikit-learn` случайные леса реализованы в классах `sklearn.ensemble.RandomForestClassifier` (для классификации) и `sklearn.ensemble.RandomForestRegressor` (для регрессии). Обучение модели производится с помощью функции `fit`, построение прогнозов — с помощью функции `predict`. Число деревьев задается с помощью поля класса `n_estimators`.

Пример использования:

In [2]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
X = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([-3, 1, 10])
clf = RandomForestRegressor(n_estimators=100)
clf.fit(X, y)
predictions = clf.predict(X)
predictions

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 2 days

    After the trial mode has expired, if you want to use mkl thereafter,
    please purchase a license at http://continuum.io
    


array([-2.03, -0.15,  7.36])

Также в этом задании вам понадобится вычислять качество предсказаний на тестовой выборке. Мы будем пользоваться метрикой $R^2$ — по сути, это среднеквадратичная ошибка (RMSE), нормированная на отрезок $[0, 1]$ и обращенная так, чтобы ее наилучшим значением была единица. Ее можно вычислить с помощью функции `sklearn.metrics.r2_score`. Первым аргументов является список правильных ответов на выборке, вторым — список предсказанных ответов. Пример использования:

In [3]:
from sklearn.metrics import r2_score
print(r2_score([10, 11, 12], [9, 11, 12.1]))

0.495


#### Загрузите данные из файла abalone.csv
Это датасет, в котором требуется предсказать возраст ракушки (число колец) по физическим измерениям.

In [4]:
import pandas as pd

df = pd.read_csv('abalone.csv')
df.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


#### Преобразуйте признак Sex в числовой
Значение F должно перейти в -1, I — в 0, M — в 1.

In [8]:
df['Sex'] = df['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0))
df.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


#### Разделите содержимое файлов на признаки и целевую переменную
В последнем столбце записана целевая переменная, в остальных — признаки.

In [9]:
X = df.iloc[:,:-1]
X.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070
2,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155
4,0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055


In [10]:
y = df['Rings']
y[:5]

0    15
1     7
2     9
3    10
4     7
Name: Rings, dtype: int64

#### Обучите случайный лес 
`sklearn.ensemble.RandomForestRegressor` - с различным числом деревьев: от 1 до 50 (не забудьте выставить `random_state=1` в конструкторе). Для каждого из вариантов оцените качество работы полученного леса на кросс-валидации по 5 блокам. Используйте параметры `random_state=1` и `shuffle=True` при создании генератора кросс-валидации `sklearn.cross_validation.KFold`. В качестве меры качества воспользуйтесь долей правильных ответов (`sklearn.metrics.r2_score`).

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import r2_score

cr_val = KFold(X.shape[0], n_folds=5, shuffle=True, random_state=1)

result=[]
for i in range(1,51):
    clf = RandomForestRegressor(n_estimators=i, random_state=1)
    res = cross_val_score(clf, X, y, cv=cr_val, scoring='r2').mean()
    result.append(res)
    
result

[0.10213869487724367,
 0.33841675515802144,
 0.40357984946186909,
 0.44272239896668103,
 0.46402076606749693,
 0.47058163275877918,
 0.47583061635230062,
 0.4817418456255852,
 0.4883478130215681,
 0.49446412480247826,
 0.4933965550001963,
 0.49796587639761541,
 0.50213646057228534,
 0.50642869622573283,
 0.50833119704321006,
 0.51051314383221469,
 0.51384829479930283,
 0.51632754127394931,
 0.51903468813638798,
 0.51867359287238224,
 0.51983542335425015,
 0.52015835367222107,
 0.52101727093662509,
 0.52240318258080909,
 0.52261746396760711,
 0.52380615817170517,
 0.52412235221889814,
 0.52505374512324399,
 0.52597037563489313,
 0.52653782166757546,
 0.52709985443060225,
 0.52838940820927838,
 0.52961655612292768,
 0.52956395544383794,
 0.52951724744047302,
 0.5295996267890184,
 0.52913141344745995,
 0.52916025763348329,
 0.52924098102743389,
 0.52906701596316297,
 0.52934200569111833,
 0.52957320653375461,
 0.52937549466485323,
 0.52911923375632508,
 0.52837546602772023,
 0.52852042754

#### Определите, при каком минимальном количестве деревьев случайный лес показывает качество на кросс-валидации выше 0.52

In [14]:
len(list(filter(lambda x:x<0.52, result))) + 1

22

#### Обратите внимание на изменение качества по мере роста числа деревьев 
Ухудшается ли оно?

In [ ]:
# Ваш код здесь